In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import HTML
import ipywidgets as wg
from IPython.display import display

In [2]:
file="State_Custom_Data.csv"
df=pd.read_csv(file, thousands=',')
#df1

In [3]:
def buildTable(data, state, period, cause):
    data1 = data.copy()
    data1.columns = data1.columns.str.strip()
    data1.loc[(data1['TOTAL DEATHS'] == 'Insufficient Data'),'TOTAL DEATHS']='0'
    data1.loc[(data1['NUM INFLUENZA DEATHS'] == 'Insufficient Data'),'NUM INFLUENZA DEATHS']='0'
    data1.loc[(data1['NUM PNEUMONIA DEATHS'] == 'Insufficient Data'),'NUM PNEUMONIA DEATHS']='0'
    data1['ORDER'] = 2
    data1.loc[(data1['WEEK'] >= 40),'ORDER']=1

    data1['TOTAL DEATHS'] = data1["TOTAL DEATHS"].astype({'TOTAL DEATHS': 'int32'})
    data1['NUM INFLUENZA DEATHS'] = data1["NUM INFLUENZA DEATHS"].astype({'NUM INFLUENZA DEATHS': 'int32'})
    data1['NUM PNEUMONIA DEATHS'] = data1["NUM PNEUMONIA DEATHS"].astype({'NUM PNEUMONIA DEATHS': 'int32'})

    #print("State:", state)
    if state == 'National':
        df2 = data1
    else:    
        df2 = data1[data['SUB AREA'] == state]
        
    if cause == 'TOTAL DEATHS':
        allCause= df2[['SEASON', 'WEEK', 'ORDER', 'TOTAL DEATHS']]
    elif cause == 'NUM INFLUENZA DEATHS':
        allCause= df2[['SEASON', 'WEEK', 'ORDER', 'NUM INFLUENZA DEATHS']]
    else:
        allCause= df2[['SEASON', 'WEEK', 'ORDER', 'NUM PNEUMONIA DEATHS']]
#    print(allCause)        
    ptable2 = pd.pivot_table(allCause, values = cause, index=['ORDER','WEEK'], columns='SEASON', aggfunc=np.sum, fill_value=0)
    ptable2.reset_index(inplace = True)

    cols = list(ptable2.columns.values)
    #print(cols)
    cols.reverse()
    #print(cols)
    cols = (cols[-2:] + cols[0:-2])
    
    ptable2 = ptable2[cols].copy()
    #print(ptable2)
    
    ptable2 = pd.DataFrame(ptable2.to_records())
    if period == 'Full Year':
        #ptable2.drop(ptable2[ptable2['WEEK'] == 53].index, inplace=True)
        ptable2 = ptable2.loc[ptable2['WEEK'] != 53]
    else:
        ptable2 = ptable2.loc[ptable2['WEEK'] != 53]
        ptable2 = ptable2.loc[ptable2['2019-20'] != 0]
        #ptable2.drop(ptable2['2019-20'] == 0, inplace=True)
        w = list(ptable2['WEEK'])
        ptable2 = ptable2.loc[(ptable2['WEEK'] < w[-1]) | (ptable2['WEEK'] >= 40)]

    ptable2 = ptable2.drop(columns="ORDER")
    
    return ptable2

def printTable(data, state = 'National', period = 'Flu Season', cause = 'TOTAL DEATHS'):
    table = buildTable(data, state, period, cause)
    table.plot(y=['2019-20', '2018-19', '2017-18', '2016-17', '2015-16', '2014-15','2013-14'],
              figsize=(12,8), title = cause)
    plt.show()
    display(cause + " " + state)
    display(HTML(table.to_html(index=False)))

#printTable(df, cause = 'NUM INFLUENZA DEATHS')



In [4]:
stateList = df.copy()
stateList = ['National'] + pd.DataFrame(stateList.groupby(['SUB AREA']).sum()).reset_index()['SUB AREA'].tolist()
#display(stateList)
areaSel = wg.Dropdown(
    options=stateList,
    value='National',
    description='Number:',
    disabled=False,
)

periodSel = wg.RadioButtons(
    options=['Flu Season', 'Full Year'],
    value='Flu Season', # Defaults to 'pineapple'
#    layout={'width': 'max-content'}, # If the items' names are long
    description='Time Period:',
    disabled=False
)

causeSel = wg.RadioButtons(
    options=['TOTAL DEATHS', 'NUM INFLUENZA DEATHS', 'NUM PNEUMONIA DEATHS'],
    value='TOTAL DEATHS', # Defaults to 'pineapple'
#    layout={'width': 'max-content'}, # If the items' names are long
    description='Time Period:',
    disabled=False
)

dummy = wg.interact(printTable, data = wg.fixed(df), state = areaSel, period = periodSel, cause = causeSel)


interactive(children=(Dropdown(description='Number:', options=('National', 'Alabama', 'Alaska', 'Arizona', 'Ar…

In [6]:
df

,AREA,SUB AREA,AGE GROUP,SEASON,WEEK,PERCENT P&I,NUM INFLUENZA DEATHS,NUM PNEUMONIA DEATHS,TOTAL DEATHS,PERCENT COMPLETE
0,State,Alabama,All,2019-20,40,5.0,0,48,955,98.3%
1,State,Alabama,All,2019-20,41,3.8,0,36,940,96.8%
2,State,Alabama,All,2019-20,42,4.5,0,45,1011,> 100%
3,State,Alabama,All,2019-20,43,4.4,2,42,1000,> 100%
4,State,Alabama,All,2019-20,44,5.7,2,53,962,99%
...,...,...,...,...,...,...,...,...,...,...
19807,State,New York City,All,2012-13,35,9.3,0,84,907,89%
19808,State,New York City,All,2012-13,36,8.7,0,78,893,87.6%
19809,State,New York City,All,2012-13,37,9.1,0,84,923,90.6%
19810,State,New York City,All,2012-13,38,9.6,0,91,947,92.9%
